In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as K
import matplotlib.pyplot as plt
import os

In [ ]:
root_dir = "C:\Users\jorda\Documents"
def load_data(path):
    train_files = []
    train_loc = os.path.join(path, "\keras_png_slices_data\keras_png_slices_train")
    for filename in os.listdir(train_loc):
        train_files.append(os.path.join(train_loc, filename))

    test_files = []
    test_loc = os.path.join(path, "\keras_png_slices_data\keras_png_slices_test")
    for filename in os.listdir(test_loc):
        test_files.append(os.path.join(test_loc, filename))

    val_files = []
    val_loc = os.path.join(path, "\keras_png_slices_data\keras_png_slices_validate")
    for filename in os.listdir(val_loc):
        val_files.append(os.path.join(val_loc, filename))        
    
    return train_files, test_files, val_files
        


In [ ]:
root_dir = "C:\Users\jorda\Documents"
directories = ["keras_png_slices_data\keras_png_slices_test",
            "keras_png_slices_data\keras_png_slices_train",
            "keras_png_slices_data\keras_png_slices_validate"]
def load_data2(path, directories, batches):
    test_imgs = K.utils.image_dataset_from_directory(os.path.join(path, directories[0]),
                                                        color_mode='greyscale',
                                                        batch_size=batches,
                                                        label_mode = None,
                                                        image_size=(256, 256))       

    train_imgs = K.utils.image_dataset_from_directory(os.path.join(path, directories[1]),
                                                        color_mode='greyscale',
                                                        label_mode = None,
                                                        batch_size=batches,
                                                        image_size=(256, 256))       

    val_imgs = K.utils.image_dataset_from_directory(os.path.join(path, directories[2]),
                                                        color_mode='greyscale',
                                                        label_mode = None,
                                                        batch_size=batches,
                                                        image_size=(256, 256))       

    
    test_imgs = test_imgs/255
    train_imgs = train_imgs/255
    val_imgs = val_imgs/255

    return test_imgs, train_imgs, val_imgs